In [ ]:
import os
import cv2
import torch
import csv
import torchvision.ops as ops
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
# ─── 0. Configuration ─────────────────────────────────────────────────────────
input_folder  = "./videos"
output_folder = "./output_videos"
os.makedirs(output_folder, exist_ok=True)

SKIP     = 1
IMG_SIZE = 512
device   = 0

# ─── 1. Prepare CSV log ───────────────────────────────────────────────────────
log_path = os.path.join(output_folder, "tracking_log.csv")
with open(log_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["video", "frame_id", "track_id", "x1", "y1", "x2", "y2"])

# ─── 2. Load models + tracker ─────────────────────────────────────────────────
head_model = YOLO("head_medium.pt")
body_model = YOLO("yolov8n.pt")
tracker    = DeepSort(max_age=30)

# ─── 3. Process videos 01.mp4 → 20.mp4 ────────────────────────────────────────
for i in range(1, 21):
    vid_name = f"{i:02}.mp4"
    in_path  = os.path.join(input_folder, vid_name)
    out_path = os.path.join(output_folder, f"tracked_{vid_name}")

    if not os.path.isfile(in_path):
        print(f"  Skipping missing {vid_name}")
        continue

    print(f"  Processing {vid_name}")
    cap = cv2.VideoCapture(in_path)
    fps    = cap.get(cv2.CAP_PROP_FPS) or 30
    w      = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h      = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out    = cv2.VideoWriter(out_path, fourcc, fps, (w, h))

    frame_id = 0
    tracks   = []  # to keep last tracks between skips

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # ─── A. Inference & tracking every SKIP frames ────────────────
        if frame_id % SKIP == 0:
            # Detect heads and bodies
            head_res = head_model(frame, conf=0.15, iou=0.5, imgsz=IMG_SIZE)[0]
            body_res = body_model(frame, conf=0.2,  iou=0.5, imgsz=IMG_SIZE)[0]

            # Collect detections: [x1,y1,x2,y2,conf,cls]
            dets = []
            for x1,y1,x2,y2,conf,cls in head_res.boxes.data.tolist():
                dets.append([x1,y1,x2,y2,conf,1])
            for x1,y1,x2,y2,conf,cls in body_res.boxes.data.tolist():
                if int(cls) == 0:
                    dets.append([x1,y1,x2,y2,conf,0])

            # NMS
            if dets:
                dt   = torch.tensor(dets)
                keep = ops.nms(dt[:,:4], dt[:,4], iou_threshold=0.5)
                final = dt[keep]
            else:
                final = torch.empty((0,6))

            # Prep for DeepSORT
            to_track = [
                ([x1, y1, x2-x1, y2-y1], float(conf), int(cls))
                for x1,y1,x2,y2,conf,cls in final.tolist()
            ]

            # Update tracker
            tracks = tracker.update_tracks(to_track, frame=frame)

        # ─── B. Draw & log every confirmed track ────────────────────────
        with open(log_path, "a", newline="") as csvfile:
            writer = csv.writer(csvfile)
            for tr in tracks:
                if not tr.is_confirmed():
                    continue
                x1, y1, x2, y2 = map(int, tr.to_ltrb())
                # draw box
                cls   = getattr(tr, "det_class", None)
                color = (255,0,0) if cls==0 else (0,255,0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                # log bbox coords
                writer.writerow([vid_name, frame_id, tr.track_id, x1, y1, x2, y2])

        # ─── C. Always write every frame ────────────────────────────────
        out.write(frame)
        frame_id += 1

    cap.release()
    out.release()
    print(f"  Saved {out_path}")

print(f"  Done! Log saved at {log_path}")


Streaming output truncated to the last 5000 lines.

0: 288x512 1 head, 640.7ms
Speed: 6.2ms preprocess, 640.7ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 2 persons, 92.3ms
Speed: 6.0ms preprocess, 92.3ms inference, 1.2ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 head, 492.6ms
Speed: 2.6ms preprocess, 492.6ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 2 persons, 89.3ms
Speed: 2.7ms preprocess, 89.3ms inference, 1.1ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 3 heads, 503.5ms
Speed: 2.5ms preprocess, 503.5ms inference, 0.9ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 person, 84.9ms
Speed: 2.9ms preprocess, 84.9ms inference, 1.2ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 3 heads, 512.4ms
Speed: 2.6ms preprocess, 512.4ms inference, 0.9ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 person, 87.9ms
Speed: 2.9ms preprocess, 87.9ms in

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
from collections import defaultdict, deque

# ─── Config ───────────────────────────────────────────────────────────────────
video_folder = "./videos"  # Folder with 01.mp4 → 20.mp4
csv_path     = "./output_videos/tracking_log.csv"
out_folder   = "./heatmap_flow_vectors"
os.makedirs(out_folder, exist_ok=True)

FLOW_MEMORY = 5          # Number of past frames to average flow over
MOTION_THRESH = 2        # Minimum movement in pixels to be considered valid flow
VECTOR_LIFE = 10         # How many frames a vector should persist on screen
FLOW_SCALE = 0.4         # Reduce motion vector length (for smoother look)

# ─── Load Data ────────────────────────────────────────────────────────────────
df_all = pd.read_csv(csv_path)

# ─── Process Videos ───────────────────────────────────────────────────────────
for i in range(1, 21):  # Change range as needed
    vid_name = f"{i:02}.mp4"
    in_path  = os.path.join(video_folder, vid_name)
    out_path = os.path.join(out_folder, f"heatmap_flow_{vid_name}")

    if not os.path.isfile(in_path):
        print(f"  Skipping missing video: {vid_name}")
        continue

    df = df_all[df_all["video"] == vid_name].copy()
    if df.empty:
        print(f"  No bounding boxes for: {vid_name}")
        continue

    cap = cv2.VideoCapture(in_path)
    if not cap.isOpened():
        print(f" Error: Cannot open video {vid_name}")
        continue

    fps     = int(cap.get(cv2.CAP_PROP_FPS))
    width   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

    heatmap = np.zeros((height, width), dtype=np.float32)
    track_history = defaultdict(deque)  # track_id → deque of (cx, cy)
    active_vectors = deque()  # persistent arrows

    frame_idx = 0
    print(f"  Processing {vid_name}...")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        df_frame = df[df["frame_id"] == frame_idx]
        flow_vectors = []

        for _, row in df_frame.iterrows():
            tid = int(row["track_id"])
            x1, y1, x2, y2 = int(row['x1']), int(row['y1']), int(row['x2']), int(row['y2'])
            x1, x2 = np.clip([x1, x2], 0, width - 1)
            y1, y2 = np.clip([y1, y2], 0, height - 1)

            if x2 > x1 and y2 > y1:
                heatmap[y1:y2, x1:x2] += 1

            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            history = track_history[tid]
            history.append((cx, cy))
            if len(history) > FLOW_MEMORY:
                history.popleft()

            if len(history) >= 2:
                x_coords, y_coords = zip(*history)
                x_avg = int(np.mean(x_coords))
                y_avg = int(np.mean(y_coords))

                dx = x_coords[-1] - x_coords[0]
                dy = y_coords[-1] - y_coords[0]
                dist = np.hypot(dx, dy)

                if dist >= MOTION_THRESH:
                    pt1 = (x_avg, y_avg)
                    pt2 = (int(x_avg + dx * FLOW_SCALE), int(y_avg + dy * FLOW_SCALE))
                    # Store (start, end, life, strength)
                    active_vectors.append((pt1, pt2, VECTOR_LIFE, dist))

        # ─── Smooth Heatmap ─────────────────────────────────────────────────────
        temp_heatmap = cv2.GaussianBlur(heatmap.copy(), (25, 25), 0)
        temp_heatmap = cv2.normalize(temp_heatmap, None, 0, 255, cv2.NORM_MINMAX)
        temp_heatmap = temp_heatmap.astype(np.uint8)
        color_heatmap = cv2.applyColorMap(temp_heatmap, cv2.COLORMAP_JET)
        overlay = cv2.addWeighted(frame, 0.5, color_heatmap, 0.5, 0)

        # ─── Draw Arrows ───────────────────────────────────────────────────────
        new_vectors = deque()
        for pt1, pt2, life, dist in active_vectors:
            if life > 0:
                fade = int(255 * (life / VECTOR_LIFE))  # Fade effect
                color = (fade, fade, fade)  # White to gray
                thickness = 1
                cv2.arrowedLine(overlay, pt1, pt2, color, thickness, tipLength=0.3)
                new_vectors.append((pt1, pt2, life - 1, dist))
        active_vectors = new_vectors

        out.write(overlay)
        frame_idx += 1

    cap.release()
    out.release()
    print(f" Saved: {out_path}")

print(" All enhanced heatmap + clean flow videos created.")


▶️  Processing 01.mp4...
✅ Saved: /content/heatmap_flow_vectors/heatmap_flow_01.mp4
▶️  Processing 02.mp4...
✅ Saved: /content/heatmap_flow_vectors/heatmap_flow_02.mp4
▶️  Processing 03.mp4...
✅ Saved: /content/heatmap_flow_vectors/heatmap_flow_03.mp4
▶️  Processing 04.mp4...
✅ Saved: /content/heatmap_flow_vectors/heatmap_flow_04.mp4
▶️  Processing 05.mp4...
✅ Saved: /content/heatmap_flow_vectors/heatmap_flow_05.mp4
▶️  Processing 06.mp4...
✅ Saved: /content/heatmap_flow_vectors/heatmap_flow_06.mp4
▶️  Processing 07.mp4...
✅ Saved: /content/heatmap_flow_vectors/heatmap_flow_07.mp4
▶️  Processing 08.mp4...
✅ Saved: /content/heatmap_flow_vectors/heatmap_flow_08.mp4
▶️  Processing 09.mp4...
✅ Saved: /content/heatmap_flow_vectors/heatmap_flow_09.mp4
▶️  Processing 10.mp4...
✅ Saved: /content/heatmap_flow_vectors/heatmap_flow_10.mp4
▶️  Processing 11.mp4...
✅ Saved: /content/heatmap_flow_vectors/heatmap_flow_11.mp4
▶️  Processing 12.mp4...
✅ Saved: /content/heatmap_flow_vectors/heatmap_flow

In [ ]:
#plot trajectories
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import os

# === CONFIGURATION ===
csv_path = './output_videos/tracking_log.csv'
video_dir = './videos'     # Folder where videos are located
output_dir = './trajectory_outputs'  # Folder to save plots and CSVs
video_prefix = ''
video_ext = '.mp4'
frame_stride = 10
movement_threshold = 30  # pixels
max_videos = 20
# =======================

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# Load tracking data
df = pd.read_csv(csv_path)

if 'video' not in df.columns:
    raise ValueError("CSV must contain a 'video' column for this to work on multiple videos.")

# Compute centers
df['cx'] = (df['x1'] + df['x2']) / 2
df['cy'] = (df['y1'] + df['y2']) / 2

# Function: Determine direction of movement
def get_direction(x0, y0, x1, y1):
    dx, dy = x1 - x0, y1 - y0
    angle = np.degrees(np.arctan2(-dy, dx)) % 360
    if 337.5 <= angle or angle < 22.5:
        return 'E'
    elif 22.5 <= angle < 67.5:
        return 'NE'
    elif 67.5 <= angle < 112.5:
        return 'N'
    elif 112.5 <= angle < 157.5:
        return 'NW'
    elif 157.5 <= angle < 202.5:
        return 'W'
    elif 202.5 <= angle < 247.5:
        return 'SW'
    elif 247.5 <= angle < 292.5:
        return 'S'
    elif 292.5 <= angle < 337.5:
        return 'SE'
    return None

# Process each video
for vid_num in range(1, max_videos + 1):
    vid_name = f'{video_prefix}{vid_num:02d}{video_ext}'
    print(f'\n Processing: {vid_name}')

    vid_df = df[df['video'] == vid_name]

    if vid_df.empty:
        print(f"  No tracking data found for {vid_name}, skipping.")
        continue

    direction_buckets = {
        'N': [], 'NE': [], 'E': [], 'SE': [],
        'S': [], 'SW': [], 'W': [], 'NW': []
    }

    final_csv_rows = []
    selected_trajectories = {}

    for track_id in vid_df['track_id'].dropna().unique():
        group = vid_df[vid_df['track_id'] == track_id].sort_values('frame_id')
        sparse_group = group[group['frame_id'] % frame_stride == 0]

        if len(sparse_group) < 2:
            continue

        coords = sparse_group[['cx', 'cy']].values
        dist = np.sum(np.linalg.norm(coords[1:] - coords[:-1], axis=1))

        if dist < movement_threshold:
            continue

        direction = get_direction(coords[0][0], coords[0][1], coords[-1][0], coords[-1][1])
        if direction:
            direction_buckets[direction].append((track_id, coords, sparse_group))

    for direction, candidates in direction_buckets.items():
        if not candidates:
            continue
        # Choose the longest distance track
        selected = max(candidates, key=lambda x: np.sum(np.linalg.norm(x[1][1:] - x[1][:-1], axis=1)))
        track_id, coords, sparse_group = selected
        selected_trajectories[track_id] = coords
        final_csv_rows.append(sparse_group)

    if not selected_trajectories:
        print(f"  No qualifying trajectories found for {vid_name}")
        continue

    # Save sparse CSV
    sparse_csv_path = os.path.join(output_dir, f'trajectory_{vid_num:02d}.csv')
    pd.concat(final_csv_rows)[['frame_id', 'track_id', 'cx', 'cy']].to_csv(sparse_csv_path, index=False)

    # Plot
    plt.figure(figsize=(10, 8))
    for track_id, points in selected_trajectories.items():
        xs, ys = points[:, 0], points[:, 1]
        plt.plot(xs, ys, label=f'ID {track_id}')
        plt.scatter(xs[-1], ys[-1], s=10)
    plt.title(f'Trajectories in {vid_name}')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.gca().invert_yaxis()
    plt.grid(True)
    plt.legend(loc='upper right', fontsize='x-small')
    plot_path = os.path.join(output_dir, f'trajectory_{vid_num:02d}.png')
    plt.savefig(plot_path)
    plt.close()

    print(f" Saved plot: {plot_path}")
    print(f" Saved CSV: {sparse_csv_path}")



 Processing: 01.mp4
 Saved plot: /content/trajectory_outputs/trajectory_01.png
 Saved CSV: /content/trajectory_outputs/trajectory_01.csv

 Processing: 02.mp4
 Saved plot: /content/trajectory_outputs/trajectory_02.png
 Saved CSV: /content/trajectory_outputs/trajectory_02.csv

 Processing: 03.mp4
 Saved plot: /content/trajectory_outputs/trajectory_03.png
 Saved CSV: /content/trajectory_outputs/trajectory_03.csv

 Processing: 04.mp4
 Saved plot: /content/trajectory_outputs/trajectory_04.png
 Saved CSV: /content/trajectory_outputs/trajectory_04.csv

 Processing: 05.mp4
 Saved plot: /content/trajectory_outputs/trajectory_05.png
 Saved CSV: /content/trajectory_outputs/trajectory_05.csv

 Processing: 06.mp4
 Saved plot: /content/trajectory_outputs/trajectory_06.png
 Saved CSV: /content/trajectory_outputs/trajectory_06.csv

 Processing: 07.mp4
 Saved plot: /content/trajectory_outputs/trajectory_07.png
 Saved CSV: /content/trajectory_outputs/trajectory_07.csv

 Processing: 08.mp4
 Saved plot: 

In [ ]:
#panic simulation
import cv2
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# ==== CONFIG ====
video_folder = './videos'
csv_path = './output_videos/tracking_log.csv'
output_folder = './panic_output_videos'
csv_output_path = './panic_simulation_output.csv'

panic_start_frame = 10
initial_panic_shift = 30
velocity_multiplier = 3
# =================

os.makedirs(output_folder, exist_ok=True)
df = pd.read_csv(csv_path)

# Ensure consistent column names
df.columns = [col.strip().lower() for col in df.columns]

# Final output records
output_records = []

# Loop through all 20 videos
for vid_num in range(1, 21):
    video_filename = f"{vid_num:02d}.mp4"
    video_path = os.path.join(video_folder, video_filename)
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f" Couldn't open video {video_filename}")
        continue

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out_path = os.path.join(output_folder, f"panic_{video_filename}")
    out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    frame_id = 0
    print(f" Processing video {video_filename}")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        new_frame = frame.copy()
        frame_data = df[(df['video'] == video_filename) & (df['frame_id'] == frame_id)]

        for _, row in frame_data.iterrows():
            if pd.isnull(row[['x1', 'y1', 'x2', 'y2']]).any():
                continue

            x1, y1, x2, y2 = map(int, [row['x1'], row['y1'], row['x2'], row['y2']])
            w, h = x2 - x1, y2 - y1
            if w <= 0 or h <= 0 or x1 < 0 or y1 < 0 or x2 > width or y2 > height:
                continue

            person_patch = frame[y1:y2, x1:x2]

            if frame_id >= panic_start_frame:
                frame_panic_intensity = frame_id - panic_start_frame + 1
                shift = initial_panic_shift + frame_panic_intensity * velocity_multiplier
                jitter = np.random.randint(-5, 6)
                new_x1 = min(width - w, x1 + shift + jitter)
            else:
                new_x1 = x1

            new_x2 = new_x1 + w
            if new_x1 < 0 or new_x2 > width:
                continue

            new_frame[y1:y2, new_x1:new_x2] = person_patch

            # Store updated row
            output_records.append([
                video_filename, frame_id, row['track_id'], new_x1, new_x2, y1, y2
            ])

        out.write(new_frame)
        frame_id += 1

    cap.release()
    out.release()
    print(f" Done: panic_{video_filename}")

# Save final output CSV
columns = ['video', 'frame_id', 'track_id', 'x1', 'x2', 'y1', 'y2']
output_df = pd.DataFrame(output_records, columns=columns)
output_df.to_csv(csv_output_path, index=False)
print(f" CSV saved to {csv_output_path}")

🚀 Processing video 01.mp4
✅ Done: panic_01.mp4
🚀 Processing video 02.mp4
✅ Done: panic_02.mp4
🚀 Processing video 03.mp4
✅ Done: panic_03.mp4
🚀 Processing video 04.mp4
✅ Done: panic_04.mp4
🚀 Processing video 05.mp4
✅ Done: panic_05.mp4
🚀 Processing video 06.mp4
✅ Done: panic_06.mp4
🚀 Processing video 07.mp4
✅ Done: panic_07.mp4
🚀 Processing video 08.mp4
✅ Done: panic_08.mp4
🚀 Processing video 09.mp4
✅ Done: panic_09.mp4
🚀 Processing video 10.mp4
✅ Done: panic_10.mp4
🚀 Processing video 11.mp4
✅ Done: panic_11.mp4
🚀 Processing video 12.mp4
✅ Done: panic_12.mp4
🚀 Processing video 13.mp4
✅ Done: panic_13.mp4
🚀 Processing video 14.mp4
✅ Done: panic_14.mp4
🚀 Processing video 15.mp4
✅ Done: panic_15.mp4
🚀 Processing video 16.mp4
✅ Done: panic_16.mp4
🚀 Processing video 17.mp4
✅ Done: panic_17.mp4
🚀 Processing video 18.mp4
✅ Done: panic_18.mp4
🚀 Processing video 19.mp4
✅ Done: panic_19.mp4
🚀 Processing video 20.mp4
✅ Done: panic_20.mp4
📁 CSV saved to panic_simulation_output.csv


In [ ]:
#heatmap for panic videos
import os
import pandas as pd
import numpy as np
import cv2
from collections import defaultdict, deque

# ─── Config ───────────────────────────────────────────────────────────────────
video_folder = "./panic_output_videos"  # Folder with 01.mp4 → 20.mp4
csv_path     = "./panic_simulation_output.csv"
out_folder   = "./panic_heatmap_flow_vectors"
os.makedirs(out_folder, exist_ok=True)

FLOW_MEMORY = 5          # Number of past frames to average flow over
MOTION_THRESH = 2        # Minimum movement in pixels to be considered valid flow
VECTOR_LIFE = 10         # How many frames a vector should persist on screen
FLOW_SCALE = 0.4         # Reduce motion vector length (for smoother look)

# ─── Load Data ────────────────────────────────────────────────────────────────
df_all = pd.read_csv(csv_path)

# ─── Process Videos ───────────────────────────────────────────────────────────
for i in range(1, 21):  # Change range as needed
    vid_name = f"panic_{i:02}.mp4"
    col_name = f"{i:02}.mp4"
    in_path  = os.path.join(video_folder, vid_name)
    out_path = os.path.join(out_folder, f"heatmap_flow_{vid_name}")

    if not os.path.isfile(in_path):
        print(f"  Skipping missing video: {vid_name}")
        continue

    df = df_all[df_all["video"] == col_name].copy()
    if df.empty:
        print(f"  No bounding boxes for: {vid_name}")
        continue

    cap = cv2.VideoCapture(in_path)
    if not cap.isOpened():
        print(f" Error: Cannot open video {vid_name}")
        continue

    fps     = int(cap.get(cv2.CAP_PROP_FPS))
    width   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

    heatmap = np.zeros((height, width), dtype=np.float32)
    track_history = defaultdict(deque)  # track_id → deque of (cx, cy)
    active_vectors = deque()  # persistent arrows

    frame_idx = 0
    print(f"  Processing {vid_name}...")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        df_frame = df[df["frame_id"] == frame_idx]
        flow_vectors = []

        for _, row in df_frame.iterrows():
            tid = int(row["track_id"])
            x1, y1, x2, y2 = int(row['x1']), int(row['y1']), int(row['x2']), int(row['y2'])
            x1, x2 = np.clip([x1, x2], 0, width - 1)
            y1, y2 = np.clip([y1, y2], 0, height - 1)

            if x2 > x1 and y2 > y1:
                heatmap[y1:y2, x1:x2] += 1

            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            history = track_history[tid]
            history.append((cx, cy))
            if len(history) > FLOW_MEMORY:
                history.popleft()

            if len(history) >= 2:
                x_coords, y_coords = zip(*history)
                x_avg = int(np.mean(x_coords))
                y_avg = int(np.mean(y_coords))

                dx = x_coords[-1] - x_coords[0]
                dy = y_coords[-1] - y_coords[0]
                dist = np.hypot(dx, dy)

                if dist >= MOTION_THRESH:
                    pt1 = (x_avg, y_avg)
                    pt2 = (int(x_avg + dx * FLOW_SCALE), int(y_avg + dy * FLOW_SCALE))
                    # Store (start, end, life, strength)
                    active_vectors.append((pt1, pt2, VECTOR_LIFE, dist))

        # ─── Smooth Heatmap ─────────────────────────────────────────────────────
        temp_heatmap = cv2.GaussianBlur(heatmap.copy(), (25, 25), 0)
        temp_heatmap = cv2.normalize(temp_heatmap, None, 0, 255, cv2.NORM_MINMAX)
        temp_heatmap = temp_heatmap.astype(np.uint8)
        color_heatmap = cv2.applyColorMap(temp_heatmap, cv2.COLORMAP_JET)
        overlay = cv2.addWeighted(frame, 0.5, color_heatmap, 0.5, 0)

        # ─── Draw Arrows ───────────────────────────────────────────────────────
        new_vectors = deque()
        for pt1, pt2, life, dist in active_vectors:
            if life > 0:
                fade = int(255 * (life / VECTOR_LIFE))  # Fade effect
                color = (fade, fade, fade)  # White to gray
                thickness = 1
                cv2.arrowedLine(overlay, pt1, pt2, color, thickness, tipLength=0.3)
                new_vectors.append((pt1, pt2, life - 1, dist))
        active_vectors = new_vectors

        out.write(overlay)
        frame_idx += 1

    cap.release()
    out.release()
    print(f" Saved: {out_path}")

print(" All enhanced heatmap + clean flow videos created.")


▶️  Processing panic_01.mp4...
✅ Saved: /content/panic_heatmap_flow_vectors/heatmap_flow_panic_01.mp4
▶️  Processing panic_02.mp4...
✅ Saved: /content/panic_heatmap_flow_vectors/heatmap_flow_panic_02.mp4
▶️  Processing panic_03.mp4...
✅ Saved: /content/panic_heatmap_flow_vectors/heatmap_flow_panic_03.mp4
▶️  Processing panic_04.mp4...
✅ Saved: /content/panic_heatmap_flow_vectors/heatmap_flow_panic_04.mp4
▶️  Processing panic_05.mp4...
✅ Saved: /content/panic_heatmap_flow_vectors/heatmap_flow_panic_05.mp4
▶️  Processing panic_06.mp4...
✅ Saved: /content/panic_heatmap_flow_vectors/heatmap_flow_panic_06.mp4
▶️  Processing panic_07.mp4...
✅ Saved: /content/panic_heatmap_flow_vectors/heatmap_flow_panic_07.mp4
▶️  Processing panic_08.mp4...
✅ Saved: /content/panic_heatmap_flow_vectors/heatmap_flow_panic_08.mp4
▶️  Processing panic_09.mp4...
✅ Saved: /content/panic_heatmap_flow_vectors/heatmap_flow_panic_09.mp4
▶️  Processing panic_10.mp4...
✅ Saved: /content/panic_heatmap_flow_vectors/heatma